# Tech Challenge Fase 2

# Aluno
Klauber Lage - RM358972

Link vídeo no Youtube: https://www.youtube.com/watch?v=m9NDA8SMadw&ab_channel=KlauberLage

Link repositório no GitHub: https://github.com/klauberfreitas/fiap/tree/main/Fase%201/Tech%20Challenge

# O PROBLEMA
O desafio consiste em projetar, implementar e testar um sistema que 
utilize Algoritmos Genéticos para otimizar uma função ou resolver um problema 
complexo de otimização. Você pode escolher problemas como otimização de 
rotas, alocação de recursos e design de redes neurais.

# Tarefas
REQUISITOS DO PROJETO

- Definição do Problema: escolha um problema real que possa ser 
resolvido por meio de otimização genética. Descreva o problema, os 
objetivos e os critérios de sucesso.

- Testes e Resultados: realize testes para demonstrar a eficácia do 
algoritmo. Compare os resultados obtidos com métodos de solução 
convencionais.

- Documentação: forneça uma documentação completa do projeto, 
incluindo descrição do problema, detalhes da implementação do 
algoritmo, análises de resultados e conclusões.


### Referência:
[Documento PDF do Desafio](POSTECH-TechChallenge-IADT-Fase1.pdf)

---



# Relatório 

Usarei a mesma estrutura do primeiro Tech Challenge, seguindo a ordem de resolução dos problemas de acordo com o pedido.

Escolhi a otimização de rotas por ser o que mais me interessou, já que talvez eu consiga aplicar isto de certa forma, em um cenário real no trabalho.

O problema: Conseguir 

Objetivos

Critérios de sucesso
- A aplicação deve rodar sem erros até o a geração definida na variável GERACOES.
- O melhor indivíduo de cada geração deve ser armazenado para ser usado nas próximas gerações.
- O melhor indivíduo deve ser usado no próximo ciclo de execução, sendo então o ponto de partida dessa execução.


# 0 - Instalação de dependências

In [5]:
# ! pip install numpy random

In [6]:
# ! pip install pygame

In [7]:
import pygame
import random
import numpy as np
import matplotlib.pyplot as plt
import itertools
import json

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

In [8]:
import random
import numpy as np
import pygame
import sys

# Definindo as cidades
num_cities = 20
cities = np.random.rand(num_cities, 2) * 100

# Parâmetros do algoritmo genético
population_size = 100
mutation_rate = 0.01
generations = 500

# Função para criar uma rota aleatória
def create_route():
    return random.sample(range(num_cities), num_cities)

# Função para criar a população inicial
def initial_population(pop_size):
    return [create_route() for _ in range(pop_size)]

# Função para calcular a distância entre duas cidades
def distance(city1, city2):
    return np.linalg.norm(cities[city1] - cities[city2])

# Função para calcular a aptidão de uma rota
def fitness(route):
    return 1 / sum(distance(route[i], route[i + 1]) for i in range(len(route) - 1))

# Função para calcular a aptidão da população
def rank_routes(population):
    return sorted([(route, fitness(route)) for route in population], key=lambda x: x[1], reverse=True)

# Função para selecionar os pais usando o método de roleta
def selection(pop_ranked):
    selection_results = []
    df = sum([pair[1] for pair in pop_ranked])
    cum_sum = np.cumsum([pair[1] / df for pair in pop_ranked])
    for _ in range(len(pop_ranked)):
        pick = random.random()
        for i, value in enumerate(cum_sum):
            if pick <= value:
                selection_results.append(pop_ranked[i][0])
                break
    return selection_results

# Função para realizar o cruzamento (crossover)
def crossover(parent1, parent2):
    start, end = sorted(random.sample(range(num_cities), 2))
    child = [None] * num_cities
    child[start:end] = parent1[start:end]
    pointer = 0
    for city in parent2:
        if city not in child:
            while child[pointer] is not None:
                pointer += 1
            child[pointer] = city
    return child

# Função para realizar a mutação
def mutate(route, mutation_rate):
    for swapped in range(len(route)):
        if random.random() < mutation_rate:
            swap_with = int(random.random() * len(route))
            route[swapped], route[swap_with] = route[swap_with], route[swapped]
    return route

# Função para criar a nova geração
def next_generation(current_gen):
    pop_ranked = rank_routes(current_gen)
    selection_results = selection(pop_ranked)
    children = []
    for i in range(0, len(selection_results) - 1, 2):
        child = crossover(selection_results[i], selection_results[i + 1])
        children.append(child)
    next_gen = [mutate(child, mutation_rate) for child in children]
    # Garantir que a nova geração tenha o mesmo tamanho da população original
    while len(next_gen) < population_size:
        next_gen.append(create_route())
    return next_gen

# Inicializando o Pygame
pygame.init()
screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Algoritmo Genético - TSP")
font = pygame.font.SysFont(None, 24)

# Função para desenhar as cidades e a rota
def draw_route(route, generation, best_distance):
    screen.fill((255, 255, 255))
    for city in cities:
        pygame.draw.circle(screen, (0, 0, 255), (int(city[0] * 8), int(city[1] * 6)), 5)
    for i in range(len(route) - 1):
        pygame.draw.line(screen, (0, 255, 0), (int(cities[route[i]][0] * 8), int(cities[route[i]][1] * 6)),
                         (int(cities[route[i + 1]][0] * 8), int(cities[route[i + 1]][1] * 6)), 2)
    text = font.render(f'Geração: {generation} | Distância: {best_distance:.2f}', True, (0, 0, 0))
    screen.blit(text, (10, 10))
    pygame.display.flip()

# Executando o algoritmo genético
population = initial_population(population_size)
best_route = None
best_distance = float('inf')

for generation in range(generations):
    population = next_generation(population)
    ranked_population = rank_routes(population)
    current_best_route = ranked_population[0][0]
    current_best_distance = 1 / ranked_population[0][1]

    if current_best_distance < best_distance:
        best_distance = current_best_distance
        best_route = current_best_route

    draw_route(best_route, generation, best_distance)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            

# Fechar o Pygame após atingir o número de gerações
pygame.quit()
